In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
import numpy as np
import os
import utils
import random as rn
import RGCN

In [2]:
SEED = 123
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
tf.random.set_seed(SEED)
np.random.seed(SEED)
rn.seed(SEED)

data = np.load(os.path.join('..','data','royalty.npz'))

entities = data['entities'].tolist()
relations = data['relations'].tolist()

NUM_ENTITIES = len(entities)
NUM_RELATIONS = len(relations)
EMBEDDING_DIM = 30
OUTPUT_DIM = 50
LEARNING_RATE = 1e-3
NUM_EPOCHS = 1
BATCH_SIZE = 1

ent2idx = dict(zip(entities, range(NUM_ENTITIES)))
rel2idx = dict(zip(relations, range(NUM_RELATIONS)))

triples, traces = data['grandmother_triples'], data['grandmother_traces']

train2idx = utils.array2idx(triples, ent2idx,rel2idx)

NUM_TRIPLES = train2idx.shape[0]

adj_mats = utils.get_adjacency_matrix_list(
    num_relations=NUM_RELATIONS,
    num_entities=NUM_ENTITIES,
    data=train2idx
)

train2idx = np.expand_dims(train2idx,axis=0)

all_indices = np.arange(NUM_ENTITIES).reshape(1,-1)

In [ ]:
# class RGCN_Layer(tf.keras.layers.Layer):
#     def __init__(self,num_relations,output_dim,**kwargs):
#         super(RGCN_Layer,self).__init__(**kwargs)
#         self.num_relations = num_relations
#         self.output_dim = output_dim
        
#     def build(self,input_shape):

#         input_dim = int(input_shape[-2][-1])
        
#         self.relation_kernel = self.add_weight(
#             shape=(self.num_relations,input_dim, self.output_dim),
#             name="relation_kernels",
#             trainable=True,
#             initializer=tf.keras.initializers.RandomNormal(
#                 mean=0.0,
#                 stddev=1,
#                 seed=SEED
#             )
#         )

#         self.self_kernel = self.add_weight(
#             shape=(input_dim, self.output_dim),
#             name="self_kernel",
#             trainable=True,
#             initializer=tf.keras.initializers.RandomNormal(
#                 mean=0.0,
#                 stddev=1,
#                 seed=SEED
#             )
#         )
    
#     def call(self, inputs):
        
#         embeddings,head_idx,head_e,tail_idx,tail_e,adj_mats = inputs
            
#         head_output = tf.matmul(head_e,self.self_kernel)
#         tail_output = tf.matmul(tail_e,self.self_kernel)
                
#         for i in range(self.num_relations):
            
#             adj_i = adj_mats[i]
            
#             head_adj = tf.nn.embedding_lookup(adj_i,head_idx)
#             tail_adj = tf.nn.embedding_lookup(adj_i,tail_idx)
            
#             head_update = tf.matmul(head_adj,embeddings)
#             tail_update = tf.matmul(tail_adj,embeddings)

#             head_output += tf.matmul(head_update,self.relation_kernel[i])
#             tail_output += tf.matmul(tail_update,self.relation_kernel[i])
       
#         return head_output, tail_output

# class DistMult(tf.keras.layers.Layer):
#     def __init__(self, num_relations,**kwargs):
#         super(DistMult,self).__init__(**kwargs)
#         self.num_relations = num_relations
        
#     def build(self,input_shape):
        
#         embedding_dim = input_shape[0][-1]
        
#         self.kernel = self.add_weight(
#             shape=(self.num_relations,embedding_dim),
#             trainable=True,
#             initializer=tf.keras.initializers.RandomNormal(
#                 mean=0.0,
#                 stddev=1,
#                 seed=SEED
#             ),
#             name='rel_embedding'
#         )
        
#     def call(self,inputs):
        
#         head_e,rel_idx,tail_e = inputs
        
#         rel_e = tf.nn.embedding_lookup(self.kernel,rel_idx)
        
#         score = tf.sigmoid(tf.reduce_sum(head_e*rel_e*tail_e,axis=-1))

#         return tf.expand_dims(score,axis=0)

# class RGCN_Model(tf.keras.Model):

#     def __init__(self,num_entities,*args,**kwargs):
#         super(RGCN_Model,self).__init__(*args, **kwargs)
#         self.num_entities = num_entities

#     def train_step(self,data):

#         all_indices,pos_head,rel,pos_tail,adj_mats = data[0]
#         y = data[1]

#         neg_head, neg_tail = utils.get_negative_triples(
#                 head=pos_head, 
#                 rel=rel, 
#                 tail=pos_tail,
#                 num_entities=self.num_entities
#             )

#         with tf.GradientTape() as tape:

#             y_pos_pred = self([
#                     all_indices,
#                     pos_head,
#                     rel,
#                     pos_tail,
#                     adj_mats
#                     ],
#                     training=True
#                 )
            
#             y_neg_pred = self([
#                     all_indices,
#                     neg_head,
#                     rel,
#                     neg_tail,
#                     adj_mats
#                     ],
#                     training=True
#                 )

#             y_pred = tf.concat([y_pos_pred,y_neg_pred],axis=1)
#             y_true = tf.concat([y,tf.zeros_like(y)],axis=1)
            
#             loss = self.compiled_loss(y_true,y_pred)

#         grads = tape.gradient(loss, self.trainable_weights)
#         self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        
#         self.compiled_metrics.update_state(y_true, y_pred)

#         return {m.name: m.result() for m in self.metrics}

In [3]:
model = RGCN.get_RGCN_Model(
    num_triples=NUM_TRIPLES,
    num_entities=NUM_ENTITIES,
    num_relations=NUM_RELATIONS,
    embedding_dim=EMBEDDING_DIM,
    output_dim=OUTPUT_DIM,
    seed=SEED
)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity t

In [4]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(), 
    optimizer=tf.keras.optimizers.SGD(learning_rate=LEARNING_RATE)
)

model.fit(
    x=[
        all_indices,
        train2idx[:,:,0],
        train2idx[:,:,1],
        train2idx[:,:,2],
        adj_mats
        ],
    y=np.ones(NUM_TRIPLES).reshape(1,-1),
    epochs=NUM_EPOCHS,
    batch_size=1,
    verbose=1
)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 0s 1ms/step - loss: 7.4756


In [ ]:
# optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
# bce = tf.keras.losses.BinaryCrossentropy()

# data = tf.data.Dataset.from_tensor_slices((
#         train2idx[0,:,0],
#         train2idx[0,:,1],
#         train2idx[0,:,2], 
#         np.ones(train2idx.shape[1])
#     )
# ).batch(BATCH_SIZE)

# for epoch in range(NUM_EPOCHS):

#     for pos_head,rel,pos_tail,y in data:

#         neg_head, neg_tail = utils.get_negative_triples(
#             head=pos_head, 
#             rel=rel, 
#             tail=pos_tail,
#             num_entities=NUM_ENTITIES
#         )

#         with tf.GradientTape() as tape:
            
# #             print(all_indices.shape)
# #             print(pos_head.shape)
# #             print(adj_mats.shape)

#             y_pos_pred = model([
#                 all_indices,
#                 pos_head,
#                 rel,
#                 pos_tail,
#                 adj_mats
#                 ],
#                 training=True
#             )

#             y_neg_pred = model([
#                 all_indices,
#                 neg_head,
#                 rel,
#                 neg_tail,
#                 adj_mats
#                 ],
#                 training=True
#             )

#             y_pred = tf.concat([y_pos_pred,y_neg_pred],axis=0)
#             y_true = tf.concat([y,tf.zeros_like(y)],axis=0)

#             loss = bce(y_true,y_pred)

#         grads = tape.gradient(loss, model.trainable_weights)
#         optimizer.apply_gradients(zip(grads, model.trainable_weights))

#     print(f'loss {loss} after epoch {epoch}')

In [5]:
print(all_indices.shape)
print(train2idx[:,:,0].shape)
print(train2idx[:,:,1].shape)
print(train2idx[:,:,2].shape)
print(adj_mats.shape)

(1, 5983)
(1, 4024)
(1, 4024)
(1, 4024)
(1, 4, 5983, 5983)


In [6]:
preds = model.predict(
    x=[
        all_indices,
        train2idx[:,:,0],
        train2idx[:,:,1],
        train2idx[:,:,2],
        adj_mats
    ]
)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [7]:
preds.shape

(1, 4024)

In [9]:
masks = tf.Variable(
    initial_value=tf.random.normal(
        (adj_mats.shape), 
        mean=0, 
        stddev=1, 
        dtype=tf.dtypes.float32, 
        seed=SEED),
    name='mask'
)

In [17]:
model([all_indices,train2idx[:,0:1,0],train2idx[:,0:1,1],train2idx[:,0:1,2],masks])

In [11]:
embeddings = model.get_layer('entity_embeddings').get_weights()[0]

In [12]:
h,_,t = train2idx[0,0,:]

k_hop_adj_mats = []

for r in range(NUM_RELATIONS):

    k_hop_adj_mat = np.zeros((NUM_ENTITIES,NUM_ENTITIES))

    head_neighbors = tf.where(adj_mats[0,r,:][train2idx[0,0:1,0]]==1.)[:,-1]
    tail_neighbors = tf.where(adj_mats[0,r,:][train2idx[0,0:1,2]]==1.)[:,-1]

    for h_i in head_neighbors:
        k_hop_adj_mat[h,h_i] = 1
        k_hop_adj_mat[h_i,h] = 1
        
    for t_i in tail_neighbors:
        k_hop_adj_mat[t,t_i] = 1
        k_hop_adj_mat[t_i,t] = 1

    k_hop_adj_mats.append(k_hop_adj_mat)

k_hop_adj_mats = np.expand_dims(k_hop_adj_mats,axis=0)

In [15]:
edge_mask = tf.cast((k_hop_adj_mats*tf.nn.sigmoid(masks)) > .1,dtype=tf.int32)

In [20]:
bce = tf.keras.losses.BinaryCrossentropy()
bce([1.],[1.])

<tf.Tensor: shape=(), dtype=float32, numpy=-0.0>

In [ ]:
# class New_RGCN_Layer(tf.keras.layers.Layer):
#     def __init__(self,num_relations,output_dim,**kwargs):
#         super(New_RGCN_Layer,self).__init__(**kwargs)
#         self.num_relations = num_relations
#         self.output_dim = output_dim
        
#     def build(self,input_shape):

#         input_dim = int(input_shape[-2][-1])
        
#         self.relation_kernel = self.add_weight(
#             shape=(self.num_relations,input_dim, self.output_dim),
#             name="relation_kernels",
#             trainable=True,
#             initializer=tf.keras.initializers.RandomNormal(
#                 mean=0.0,
#                 stddev=1,
#                 seed=SEED
#             )
#         )


#         self.self_kernel = self.add_weight(
#             shape=(input_dim, self.output_dim),
#             name="self_kernel",
#             trainable=True,
#             initializer=tf.keras.initializers.RandomNormal(
#                 mean=0.0,
#                 stddev=1,
#                 seed=SEED
#             )
#         )
    
#     def call(self, inputs):
        
#         embeddings,head_idx,head_e,tail_idx,tail_e,adj_mats = inputs
        
# #         print('embeddings',embeddings.shape)
# #         print('head_idx',head_idx.shape)
# #         print('head_e',head_e.shape)
# #         print('adj_mats',adj_mats.shape)
            
#         head_output = tf.matmul(head_e,self.self_kernel)
#         tail_output = tf.matmul(tail_e,self.self_kernel)
        
#         #print('head_output',head_output.shape)
        
#         for i in range(self.num_relations):
            
#             adj_i = adj_mats[i]

#             #print('adj_i',adj_i.shape)
            
#             head_adj = tf.nn.embedding_lookup(adj_i,head_idx)
#             tail_adj = tf.nn.embedding_lookup(adj_i,tail_idx)
            
#             #print('head_adj',head_adj.shape)
            
#             #print('head_adj',head_adj.shape)
#             #print('embeddings',embeddings.shape)
            
#             head_update = tf.matmul(head_adj,embeddings)
#             tail_update = tf.matmul(tail_adj,embeddings)

#             head_output += tf.matmul(head_update,self.relation_kernel[i])
#             tail_output += tf.matmul(tail_update,self.relation_kernel[i])
       
#         return head_output, tail_output
    
# class DistMult(tf.keras.layers.Layer):
#     def __init__(self, num_relations,**kwargs):
#         super(DistMult,self).__init__(**kwargs)
#         self.num_relations = num_relations
        
#     def build(self,input_shape):
        
#         embedding_dim = input_shape[0][-1]
        
#         self.kernel = self.add_weight(
#             shape=(self.num_relations,embedding_dim),
#             trainable=True,
#             initializer=tf.keras.initializers.RandomNormal(
#                 mean=0.0,
#                 stddev=1,
#                 seed=SEED
#             ),
#             name='rel_embedding'
#         )
        
#     def call(self,inputs):
        
#         head_e,rel_idx,tail_e = inputs
        
#         rel_e = tf.nn.embedding_lookup(self.kernel,rel_idx)
        
#         score = tf.sigmoid(tf.reduce_sum(head_e*rel_e*tail_e,axis=-1))
        
#         return tf.expand_dims(score,axis=0)
#         embeddings,head_idx,tail_idx,head_e,tail_e,adj_mats = inputs

#         adj_mats = tf.squeeze(adj_mats,axis=0)
#         embeddings = tf.squeeze(embeddings,axis=0)

#         head_output = tf.matmul(head_e,self.self_kernel)
#         tail_output = tf.matmul(tail_e,self.self_kernel)
        
#         for i in range(self.num_relations):
            
#             adj_i = adj_mats[i]

#             head_adj = tf.nn.embedding_lookup(adj_i,head_idx)
#             tail_adj = tf.nn.embedding_lookup(adj_i,tail_idx)
            
#             h_head = tf.matmul(head_adj,embeddings)
#             h_tail = tf.matmul(head_adj,embeddings)
            
#             head_output += tf.matmul(h_head,self.relation_kernel[i])
#             tail_output += tf.matmul(h_tail,self.relation_kernel[i])

#         return head_output,tail_output

In [ ]:
# data = tf.data.Dataset.from_tensor_slices((
#         train2idx[:,:,0],
#         train2idx[:,:,1],
#         train2idx[:,:,2], 
#         np.ones(train2idx.shape[1]).reshape(1,-1)
#     )
# ).batch(1)

# for h,r,t,y in data:

#     neg_head, neg_tail = utils.get_negative_triples(head=h,rel=r,tail=t,num_entities=NUM_ENTITIES)
#     print(h)
#     print(neg_head)
#     print(t) 
#     print(neg_tail)
#     break

In [ ]:
# all_entities = tf.keras.Input(shape=(NUM_ENTITIES,), name='all_entities',dtype=tf.int64)
# head_input = tf.keras.Input(shape=(None,), name='head_input',batch_size=1,dtype=tf.int64)
# rel_input = tf.keras.Input(shape=(None,), name='rel_input',batch_size=1,dtype=tf.int64)
# tail_input = tf.keras.Input(shape=(None,), name='tail_input',batch_size=1,dtype=tf.int64)

# adj_inputs = tf.keras.Input(
#         shape=(
#             NUM_RELATIONS,
#             NUM_ENTITIES,
#             NUM_ENTITIES
#         ),
#         dtype=tf.float32,
#         name='adj_inputs'
#     )

# entity_embeddings = Embedding(
#         input_dim=NUM_ENTITIES,
#         output_dim=EMBEDDING_DIM,
#         name='entity_embeddings',
#         embeddings_initializer=tf.keras.initializers.RandomUniform(
#             minval=-1,
#             maxval=1,
#             seed=SEED
#         )
#     )

# all_e = entity_embeddings(all_entities)
# head_e = entity_embeddings(head_input)
# tail_e = entity_embeddings(tail_input)

# all_e = tf.keras.layers.Lambda(lambda x:x[0,:,:])(all_e)
# head_e = tf.keras.layers.Lambda(lambda x:x[0,:,:])(head_e)
# tail_e = tf.keras.layers.Lambda(lambda x:x[0,:,:])(tail_e)

# head_index = tf.keras.layers.Lambda(lambda x:x[0,:])(head_input)
# rel_index = tf.keras.layers.Lambda(lambda x:x[0,:])(rel_input)
# tail_index = tf.keras.layers.Lambda(lambda x:x[0,:])(tail_input)

# adj_mats_layer = tf.keras.layers.Lambda(lambda x:x[0,:,:])(adj_inputs)
# #embeddings,head_idx,head_e,tail_idx,tail_e,adj_mats

# new_head,new_tail = New_RGCN_Layer(NUM_RELATIONS,OUTPUT_DIM)([all_e,head_index,head_e,tail_index,tail_e,adj_mats_layer])
# #new_head = New_RGCN_Layer(NUM_RELATIONS,OUTPUT_DIM)([all_entities,])

# output = DistMult(num_relations=NUM_RELATIONS,name='output')([new_head,rel_index,new_tail])

# output = tf.keras.layers.Dense(train2idx.shape[1],activation='sigmoid')(output)

# #output = tf.keras.layers.Dense(1)(output)
# #output = tf.keras.layers.Reshape((1,))(output)
# # m = tf.keras.Model([head_input],[out])
# # m([train2idx[:,0:32,0]])
# m = tf.keras.Model([all_entities,head_input,rel_input,tail_input,adj_inputs],[output])

# m([np.arange(NUM_ENTITIES).reshape(1,-1),train2idx[:,:,0],train2idx[:,:,1],train2idx[:,:,2],adj_mats])

In [ ]:
# m.compile(
#     loss=tf.keras.losses.BinaryCrossentropy(), 
#     optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3)
# )

# #m.summary()
# m.fit(x=[
#     all_indices,
#     train2idx[:,:,0],
#     train2idx[:,:,1],
#     train2idx[:,:,2],
#     adj_mats
# ],
#     y=np.ones(train2idx.shape[1]).reshape(1,-1),
#     epochs=NUM_EPOCHS,
#     batch_size=1,
#     verbose=1
# )

In [ ]:
# preds = m.predict(
#     x=[
#         all_indices,
#         train2idx[:,:,0],
#         train2idx[:,:,1],
#         train2idx[:,:,2],
#         adj_mats
#     ]
# )

In [ ]:
#m([all_indices,train2idx[:,0:1,0],train2idx[:,0:1,1],train2idx[:,0:1,2],masks])

In [ ]:
#embeddings_model = tf.keras.Model(inputs=m.input,outputs=m.get_layer('entity_embeddings').output)

#embeddings = m.get_layer('entity_embeddings').get_weights()[0]

In [ ]:
# head_embeddings,tail_embeddings = embeddings_model([
#     all_indices,
#     train2idx[:,0:1,0],
#     train2idx[:,0:1,1],
#     train2idx[:,0:1,2],
#     adj_mats
# ])

In [ ]:
#train2idx[0,:,:][0:5]

In [ ]:
#tf.where(adj_mats[0][2][100]==1.).numpy()
#np.argwhere(train2idx[0,:,2]== 100)

# for h,_,t in train2idx[:,822,:]:
    
#     head_neighbors = tf.where(adj_mats[0][2][h]==1.).numpy()
#     tail_neighbors = tf.where(adj_mats[0][2][t]==1.).numpy()
#tf.where(adj_mats[0][2][100]==1.).numpy()
#tf.where(adj_mats[0,2,[100,100,100]]==1.)[:,-1]
#train2idx[:,822,[0,2]]

In [ ]:
#tf.nn.embedding_lookup(embeddings,train2idx[0,0:1,0])
#k_hop_adj_mats*tf.nn.sigmoid(masks)

In [ ]:
#for triple in test set:
    #get k hop subgraph of each triple (all neighbors of head/tail)
    #compute adjacency matrix of subgraph (1,NUM_RELATIONS,NUM_ENTITIES,NUM_ENTITIES)
    #compute pred of triple
    #for i in iter:
        #learn mask
        #mask * adj matrix 
    #mask * adj matrix-> reduce output to ints
#edge_mask = tf.cast((k_hop_adj_mats*tf.nn.sigmoid(masks)) > .1,dtype=tf.int32)
#tf.nn.sigmoid(masks)

In [ ]:
head_indices = tf.where(edge_mask[0,2,h,:] == 1)
tail_indices = tf.where(edge_mask[0,2,t,:] == 1)
# swap = tf.concat([tf.reshape(indices[:,1],(-1,1)),tf.reshape(indices[:,0],(-1,1))],axis=1)

# unique_indices = tf.concat([indices,swap],axis=0)